# PRMT-2414: Look at spread of EMIS spike errors across practices

## Context

We have learnt from the CIS team that the EMIS spikes are down to individual practices not being on Windows 10. We want to do some data analysis to understand the spread of the errors we’re seeing and see if it’s the same practices responsible for the different scenarios. We will then compare this to a list EMIS will provide to us of the practices that aren’t on Windows 10. 

### Scope
Look at the spread of the below error scenarios across practices. 

Questions to answers:

List of practices nationally and the occurances of each error, split down by the last three months 

We would like to know if it is the same practices responsible for each error codes

### Error scenarios
EMIS sending: Core extract not sent - error code 20 sender error

EMIS - EMIS: final error 25

EMIS sending or receiving: COPCs not acknowledged - no error code

EMIS to TPP: final error 31

EMIS sending: Core extract not sent - sender error 19

EMIS sending: Request not acknowledged - no error

EMIS sending: Contains fatal sender error - sender error 14

In [ ]:
import pandas as pd 
import numpy as np
import paths
from data.practice_metadata import read_asid_metadata

In [ ]:
asid_lookup=read_asid_metadata("prm-gp2gp-ods-metadata-preprod", "v2/2021/10/organisationMetadata.json")

transfer_file_location = "s3://prm-gp2gp-transfer-data-preprod/v5/2021/"

transfer_files = [
    "6/2021-6-transfers.parquet",
    "7/2021-7-transfers.parquet",
    "8/2021-8-transfers.parquet",
    "9/2021-9-transfers.parquet",
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]

transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))

transfers = transfers_raw\
    .join(asid_lookup.add_prefix("requesting_"), on="requesting_practice_asid", how="left")\
    .join(asid_lookup.add_prefix("sending_"), on="sending_practice_asid", how="left")\

transfers['month']=transfers['date_requested'].dt.to_period('M')

# Supplier name mapping
supplier_renaming = {
    "SystmOne":"TPP",
    None: "Unknown"
}

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["sending_practice_name"] = transfers["sending_practice_name"].fillna("Unknown")
transfers["requesting_practice_name"] = transfers["requesting_practice_name"].fillna("Unknown")
transfers["sending_practice_ods_code"] = transfers["sending_practice_ods_code"].fillna("Unknown")
transfers["requesting_practice_ods_code"] = transfers["requesting_practice_ods_code"].fillna("Unknown")

In [ ]:
monthly_transfers_per_sending_practice = transfers.pivot_table(
        index=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"], 
        columns="month", 
        values="conversation_id", 
        aggfunc="count"
)

monthly_transfers_per_requesting_practice = transfers.pivot_table(
        index=["requesting_practice_name", "requesting_supplier", "requesting_practice_ods_code"], 
        columns="month", 
        values="conversation_id", 
        aggfunc="count"
)


def generate_error_scenario_overview(monthy_transfers_with_error_scenario, error_scenario_name):

    monthly_error_scenario_overview = (
        monthy_transfers_with_error_scenario
            .rename(
                 {
                "2021-06_x" : "Total transfers: Jun", 
                 "2021-07_x" : "Total transfers: Jul", 
                 "2021-08_x" : "Total transfers: Aug",
                 "2021-09_x" : "Total transfers: Sept",
                 "2021-06_y" : f"Transfers with {error_scenario_name}: Jun",
                 "2021-07_y" : f"Transfers with {error_scenario_name}: Jul",
                 "2021-08_y" : f"Transfers with {error_scenario_name}: Aug",
                 "2021-09_y" : f"Transfers with {error_scenario_name}: Sept"}, axis=1)
            .reset_index()
    )
    
    monthly_error_scenario_overview["% of transfers in Jun"] = (
            (monthly_error_scenario_overview[f"Transfers with {error_scenario_name}: Jun"] / monthly_error_scenario_overview["Total transfers: Jun"])
            .multiply(100)
            .round(2)
    )

    monthly_error_scenario_overview["% of transfers in Jul"] = (
            (monthly_error_scenario_overview[f"Transfers with {error_scenario_name}: Jul"] / monthly_error_scenario_overview["Total transfers: Jul"])
            .multiply(100)
            .round(2)
    )

    monthly_error_scenario_overview["% of transfers in Aug"] = (
            (monthly_error_scenario_overview[f"Transfers with {error_scenario_name}: Aug"] / monthly_error_scenario_overview["Total transfers: Aug"])
            .multiply(100)
            .round(2)
    )

    monthly_error_scenario_overview["% of transfers in Sept"] = (
            (monthly_error_scenario_overview[f"Transfers with {error_scenario_name}: Sept"] / monthly_error_scenario_overview["Total transfers: Sept"])
            .multiply(100)
            .round(2)
    )

    return monthly_error_scenario_overview.fillna(0).sort_values(by="% of transfers in Sept", ascending=False)

### EMIS sending: Core extract not sent - error code 20 sender error

In [ ]:
emis_sender_bool = transfers["sending_supplier"] == "EMIS"
core_extract_not_sent_bool = transfers["failure_reason"] == "Core extract not sent"
sender_error_code_20_bool = transfers["sender_error_codes"].apply(lambda error_codes: 20 in error_codes)
emis_transfers_with_error_20 = transfers[emis_sender_bool & core_extract_not_sent_bool & sender_error_code_20_bool].copy()

monthly_emis_transfers_with_error_20 = emis_transfers_with_error_20.pivot_table(
        index=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_error_20_scenario_overview = monthly_transfers_per_sending_practice.merge(
        monthly_emis_transfers_with_error_20, 
        how="right", 
        on=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"]
)

monthly_error_20_sender_overview = generate_error_scenario_overview(monthly_error_20_scenario_overview, "error code 20")
monthly_error_20_sender_overview

### EMIS - EMIS: final error 25 from sender

In [ ]:
emis_sender_bool = transfers["sending_supplier"] == "EMIS"
emis_requester_bool = transfers["requesting_supplier"] == "EMIS"
final_error_code_25_bool = transfers["final_error_codes"].apply(lambda error_codes: 25 in error_codes)
emis_transfers_with_final_error_25 = transfers[emis_sender_bool & emis_requester_bool & final_error_code_25_bool].copy()

monthly_emis_transfers_with_final_error_25_sender = emis_transfers_with_final_error_25.pivot_table(
        index=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_error_25_sender_scenario_overview = monthly_transfers_per_sending_practice.merge(
        monthly_emis_transfers_with_final_error_25_sender, 
        how="right", 
        on=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"]
)

monthly_error_25_sender_overview = generate_error_scenario_overview(monthly_error_25_sender_scenario_overview, "error code 25")
monthly_error_25_sender_overview

### EMIS - EMIS: final error 25 from requester

In [ ]:
monthly_emis_transfers_with_final_error_25_requester = emis_transfers_with_final_error_25.pivot_table(
        index=["requesting_practice_name", "requesting_supplier", "requesting_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_error_25_requester_scenario_overview = monthly_transfers_per_requesting_practice.merge(
        monthly_emis_transfers_with_final_error_25_requester, 
        how="right", 
        on=["requesting_practice_name", "requesting_supplier", "requesting_practice_ods_code"]
)

monthly_error_25_requester_overview = generate_error_scenario_overview(monthly_error_25_requester_scenario_overview, "error code 25")
monthly_error_25_requester_overview

### EMIS to any : COPCs not acknowledged - no error code - sender

In [ ]:
copc_failure_reason_bool = transfers["failure_reason"] == "COPC(s) not acknowledged"

# The list may contain some elements that are NaN - which would mean there was an ack without error. 
# We need to interate through the list and check there are no numbers (error codes).

no_final_error_code_bool = transfers["final_error_codes"].apply(lambda error_codes: all(pd.isna(error) for error in error_codes))
no_sender_error_code_bool = transfers["sender_error_codes"].apply(lambda error_codes: all(pd.isna(error) for error in error_codes))
no_intermediate_error_code_bool = transfers["intermediate_error_codes"].apply(lambda error_codes: all(pd.isna(error) for error in error_codes))

emis_transfers_with_copc_not_acked = transfers[emis_sender_bool & copc_failure_reason_bool & no_final_error_code_bool & no_sender_error_code_bool & no_intermediate_error_code_bool].copy()

monthly_emis_transfers_with_copc_not_acked_sender = emis_transfers_with_copc_not_acked.pivot_table(
        index=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_copc_not_acked_sender_scenario_overview = monthly_transfers_per_sending_practice.merge(
        monthly_emis_transfers_with_copc_not_acked_sender, 
        how="right", 
        on=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"]
)

monthly_copc_not_acked_sender_overview = generate_error_scenario_overview(monthly_copc_not_acked_sender_scenario_overview, "COPC not acked")
monthly_copc_not_acked_sender_overview

### Any to EMIS : COPCs not acknowledged - no error code - requester

In [ ]:
emis_transfers_with_copc_not_acked_requester = transfers[emis_requester_bool & copc_failure_reason_bool & no_final_error_code_bool & no_sender_error_code_bool & no_intermediate_error_code_bool].copy()

monthly_emis_transfers_with_copc_not_acked_requester = emis_transfers_with_copc_not_acked_requester.pivot_table(
        index=["requesting_practice_name", "requesting_supplier", "requesting_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_copc_not_acked_requester_scenario_overview = monthly_transfers_per_requesting_practice.merge(
        monthly_emis_transfers_with_copc_not_acked_requester, 
        how="right", 
        on=["requesting_practice_name", "requesting_supplier", "requesting_practice_ods_code"]
)

monthly_copc_not_acked_requester_overview = generate_error_scenario_overview(monthly_copc_not_acked_requester_scenario_overview, "COPC not acked")
monthly_copc_not_acked_requester_overview

### EMIS to TPP: final error 31

In [ ]:
final_error_code_31_bool = transfers["final_error_codes"].apply(lambda error_codes: 31 in error_codes)
tpp_requester_bool = transfers["requesting_supplier"] == "TPP"
emis_to_tpp_transfers_with_error_31 = transfers[emis_sender_bool & tpp_requester_bool & final_error_code_31_bool].copy()

monthly_transfers_with_error_31 = emis_to_tpp_transfers_with_error_31.pivot_table(
        index=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_error_31_sender_scenario_overview = monthly_transfers_per_sending_practice.merge(
        monthly_transfers_with_error_31, 
        how="right", 
        on=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"]
)

monthly_error_31_sender_overview = generate_error_scenario_overview(monthly_error_31_sender_scenario_overview, "Error 31")
monthly_error_31_sender_overview

### EMIS sending: Core extract not sent - sender error 19

In [ ]:
sender_error_code_19_bool = transfers["sender_error_codes"].apply(lambda error_codes: 19 in error_codes)

emis_transfers_with_error_19 = transfers[emis_sender_bool & sender_error_code_19_bool].copy()

monthly_transfers_with_error_19 = emis_transfers_with_error_19.pivot_table(
        index=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_error_19_sender_scenario_overview = monthly_transfers_per_sending_practice.merge(
        monthly_transfers_with_error_19, 
        how="right", 
        on=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"]
)

monthly_error_19_sender_overview = generate_error_scenario_overview(monthly_error_19_sender_scenario_overview, "Error 19")
monthly_error_19_sender_overview

### EMIS sending: Request not acknowledged - no error

In [ ]:
request_not_acked_bool = transfers["failure_reason"] == "Request not acknowledged"
emis_transfers_request_not_acked = transfers[emis_sender_bool & request_not_acked_bool & no_final_error_code_bool & no_sender_error_code_bool & no_intermediate_error_code_bool].copy()



monthly_transfers_with_request_not_acked = emis_transfers_request_not_acked.pivot_table(
        index=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_request_not_acked_sender_scenario_overview = monthly_transfers_per_sending_practice.merge(
        monthly_transfers_with_request_not_acked, 
        how="right", 
        on=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"]
)

monthly_request_not_acked_sender_overview = generate_error_scenario_overview(monthly_request_not_acked_sender_scenario_overview, "Request not acked")
monthly_request_not_acked_sender_overview

### EMIS sending: Contains fatal sender error - sender error 14

In [ ]:
sender_error_code_14_bool = transfers["sender_error_codes"].apply(lambda error_codes: 14 in error_codes)
emis_transfers_with_error_14 = transfers[emis_sender_bool & sender_error_code_14_bool].copy()

monthly_transfers_with_error_14 = emis_transfers_with_error_14.pivot_table(
        index=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"], 
        columns="month", values="conversation_id", 
        aggfunc="count"
)

monthly_error_14_sender_scenario_overview = monthly_transfers_per_sending_practice.merge(
        monthly_transfers_with_error_14, 
        how="right", 
        on=["sending_practice_name", "sending_supplier", "sending_practice_ods_code"]
)

monthly_error_14_sender_overview = generate_error_scenario_overview(monthly_error_14_sender_scenario_overview, "Error 14")
monthly_error_14_sender_overview

In [ ]:
with pd.ExcelWriter("Emis Error Code Scenarios by Practice PRMT-2414.xlsx") as writer:
    monthly_error_20_sender_overview.to_excel(writer, sheet_name="EMIS-Any, Err 20, Sender",index=False)
    monthly_error_25_sender_overview.to_excel(writer, sheet_name="EMIS-EMIS, Err 25, Sender",index=False)
    monthly_error_25_requester_overview.to_excel(writer, sheet_name="EMIS-EMIS, Err 25, Requester",index=False)
    monthly_copc_not_acked_sender_overview.to_excel(writer, sheet_name="EMIS-Any, COPC not acked,Send",index=False)
    monthly_copc_not_acked_requester_overview.to_excel(writer, sheet_name="Any-EMIS, COPC not acked, Req",index=False)
    monthly_error_31_sender_overview.to_excel(writer, sheet_name="EMIS-TPP, Err 31, Sender",index=False)
    monthly_error_19_sender_overview.to_excel(writer, sheet_name="EMIS-Any, Err 19, Sender",index=False)
    monthly_request_not_acked_sender_overview.to_excel(writer, sheet_name="EMIS-Any, Req not acked, Sender",index=False)
    monthly_error_14_sender_overview.to_excel(writer, sheet_name="EMIS-Any, Err 14, Sender",index=False)